In [84]:
#############################################
# 1. SETUP AND READ DATA
#############################################

import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Paths to your CSV files
sales_path = '/content/filtered_drinks_data.csv'
weather_path = '/content/merged_omzet_weer_ticket.csv'

# Read data
sales_df = pd.read_csv(sales_path)
event_weather_df = pd.read_csv(weather_path)

print("Sales Data Sample:")
print(sales_df.head())
print("\nEvent & Weather Data Sample:")
print(event_weather_df.head())

#############################################
# 2. PREPROCESS & BUILD FINAL_DF
#############################################

# (A) Parse datetime columns
sales_df['Datum'] = pd.to_datetime(sales_df['Datum'], dayfirst=True)
event_weather_df['first_event_date_start'] = pd.to_datetime(event_weather_df['first_event_date_start'])

# Create event_end = start + 8 hours
event_weather_df['event_end'] = event_weather_df['first_event_date_start'] + pd.Timedelta(hours=8)

# Classify event as "day" or "night" based on start time
def classify_event_type(dt):
    hour = dt.hour
    if 14 <= hour < 18:
        return "day"
    else:
        return "night"

event_weather_df['event_type'] = event_weather_df['first_event_date_start'].apply(classify_event_type)


# Before applying agg_funcs, convert ALL relevant columns to numeric:
event_weather_df['Tmax'] = pd.to_numeric(event_weather_df['Tmax'].str.replace(' °C', ''), errors='coerce')
event_weather_df['Tmin'] = pd.to_numeric(event_weather_df['Tmin'].str.replace(' °C', ''), errors='coerce')
event_weather_df['Neerslag'] = pd.to_numeric(event_weather_df['Neerslag'].str.replace(',', '.').str.replace(' mm', ''), errors='coerce') # This line is crucial
event_weather_df['Max Windstoot'] = pd.to_numeric(event_weather_df['Max Windstoot'].str.extract('(\d+\.?\d*)', expand=False), errors='coerce')


# (B) Deduplicate by grouping event_weather_df so each event_start is unique
agg_funcs = {
    'Tmax': 'mean',
    'Tmin': 'mean',
    'Neerslag': 'sum',
    'Max Windstoot': 'max',
    'Omzet': 'sum',
    'aantal_tickets': 'max',
    'aantal_opgedaagd': 'max'
}


# Now proceed with the groupby and agg:
event_weather_dedup = (
    event_weather_df
    .groupby('first_event_date_start', as_index=False)
    .agg(agg_funcs)
)



event_weather_dedup = (
    event_weather_df
    .groupby('first_event_date_start', as_index=False)
    .agg(agg_funcs)
)

# Re-apply event_end (8h from start)
event_weather_dedup['event_end'] = event_weather_dedup['first_event_date_start'] + pd.Timedelta(hours=8)

# If multiple rows had different event_type, pick the first (or majority)
def pick_event_type(series):
    return series.iloc[0]

event_type_map = (
    event_weather_df
    .groupby('first_event_date_start')['event_type']
    .apply(pick_event_type)
    .reset_index()
)

event_weather_dedup = pd.merge(
    event_weather_dedup,
    event_type_map,
    on='first_event_date_start',
    how='left'
)

# (C) Build final_df by merging sales within each 8-hour window
final_rows = []
for _, event_row in event_weather_dedup.iterrows():
    start_time = event_row['first_event_date_start']
    end_time   = event_row['event_end']

    # Sales within [start_time, end_time)
    mask = (sales_df['Datum'] >= start_time) & (sales_df['Datum'] < end_time)
    sales_in_window = sales_df.loc[mask]

    grouped = sales_in_window.groupby('Product', as_index=False)['Aantal'].sum()
    if not grouped.empty:
        pivoted = grouped.set_index('Product')['Aantal'].to_frame().T
        pivoted_data = pivoted.iloc[0].to_dict()
    else:
        pivoted_data = {}

    # Add event/weather fields
    pivoted_data['event_start'] = start_time
    pivoted_data['event_end']   = end_time
    pivoted_data['event_type']  = event_row['event_type']
    pivoted_data['Tmax']        = event_row['Tmax']
    pivoted_data['Tmin']        = event_row['Tmin']
    pivoted_data['Neerslag']    = event_row['Neerslag']
    pivoted_data['Max Windstoot'] = event_row['Max Windstoot']
    pivoted_data['aantal_tickets']   = event_row['aantal_tickets']
    pivoted_data['aantal_opgedaagd'] = event_row['aantal_opgedaagd']
    pivoted_data['Omzet']           = event_row['Omzet']

    final_rows.append(pivoted_data)

final_df = pd.DataFrame(final_rows)
print("final_df shape:", final_df.shape)

#############################################
# 3. CLEAN AND FEATURE ENGINEERING
#############################################

# Remove trailing spaces in columns
final_df.columns = [col.strip() for col in final_df.columns]

# Identify your main drink columns
drink_columns = [
    'Buitenlands mix','Desperados','Fris','Heineken',
    'Heineken 0.0','RedBull','Rosé','Sauvignon','Shot',
    'Shot tequila','Sterk Mix','Amaretto Sour','Dark & Stormy',
    'Mojito','Moscow Mule','Pornstar Martini','Strawberry Daiquiri',
    'Virgin Cocktail','ANNA Daiquiri','Jäger-Mule'
]

# Fill missing sales with 0
final_df[drink_columns] = final_df[drink_columns].fillna(0)

# 3.2 (A) Add Season Feature
def get_season(date):
    m = date.month
    if m in [12, 1, 2]:
        return "winter"
    elif m in [3, 4, 5]:
        return "spring"
    elif m in [6, 7, 8]:
        return "summer"
    else:
        return "autumn"

final_df['season'] = final_df['event_start'].apply(get_season)

# 3.2 (B) Add Day-of-Week Feature
final_df['day_of_week'] = final_df['event_start'].dt.day_name()

# One-hot encode event_type, season, day_of_week
final_df = pd.get_dummies(final_df, columns=['event_type','season','day_of_week'])

# Check the columns now
print("Columns after feature engineering:", final_df.columns.tolist())

#############################################
# 4. SPLIT DRINKS INTO HIGH-VOLUME VS MID/LOW-VOLUME
#############################################

# Sum total sales for each product
product_sums = final_df[drink_columns].sum()
print("\nTotal sales per product:\n", product_sums)

# Decide thresholds
HIGH_THRESHOLD = 10_000
LOW_THRESHOLD  = 200  # for demonstration, might drop <200 entirely

high_volume_drinks = product_sums[product_sums >= HIGH_THRESHOLD].index.tolist()
mid_low_drinks = product_sums[(product_sums >= LOW_THRESHOLD) & (product_sums < HIGH_THRESHOLD)].index.tolist()
very_low_drinks = product_sums[product_sums < LOW_THRESHOLD].index.tolist()

print("\nHigh Volume Drinks:", high_volume_drinks)
print("Mid/Low Volume Drinks:", mid_low_drinks)
print("Dropping Very Low Volume Drinks:", very_low_drinks)

# Drop extremely low columns
final_df.drop(columns=very_low_drinks, inplace=True, errors='ignore')

# Re-check your final drink lists
drink_columns_high = high_volume_drinks
drink_columns_mid_low = mid_low_drinks

print("\nHigh Volume Model will predict:", drink_columns_high)
print("Mid/Low Volume Model will predict:", drink_columns_mid_low)






def remove_outliers_iqr(df, columns, multiplier=3):
    """
    Removes rows in df where any of the specified columns
    have values outside [Q1 - multiplier*IQR, Q3 + multiplier*IQR].

    Returns a cleaned DataFrame.
    """
    outlier_indices = set()

    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR

        # Get indices where the column value is outside the bounds
        col_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
        outlier_indices.update(col_outliers)

    print(f"Removing {len(outlier_indices)} outlier rows based on IQR method.")
    df_cleaned = df.drop(index=outlier_indices).reset_index(drop=True)
    return df_cleaned

# Identify your drink columns
drink_columns = [
    'Buitenlands mix','Desperados','Fris','Heineken',
    'Heineken 0.0','RedBull','Rosé','Sauvignon','Shot',
    'Shot tequila','Sterk Mix','Amaretto Sour','Dark & Stormy',
    'Mojito','Moscow Mule','Pornstar Martini','Strawberry Daiquiri',
    'Virgin Cocktail'
]

# 1. Outlier Removal
final_df = remove_outliers_iqr(final_df, drink_columns, multiplier=3)
print("New shape after outlier removal:", final_df.shape)







#############################################
# 5. TRAIN TWO SEPARATE MODELS
#############################################

# Common feature columns (adjust as needed)
# Must exist in final_df.columns
feature_columns = [
    'Tmax','Tmin','Neerslag','Max Windstoot',
    'aantal_tickets','aantal_opgedaagd','Omzet',
    # One-hot columns for event_type (e.g. event_type_day, event_type_night)
    # plus the season_ and day_of_week_ columns
]
# Let's auto-collect the dummies for event_type_, season_, day_of_week_
one_hot_cols = [col for col in final_df.columns if col.startswith('event_type_')
                or col.startswith('season_') or col.startswith('day_of_week_')]
feature_columns += one_hot_cols

# (A) High-Volume Model
print("\n===== HIGH VOLUME MODEL =====")
model_df_high = final_df.dropna(subset=feature_columns).copy()
X_high = model_df_high[feature_columns].values
Y_high = model_df_high[drink_columns_high].values

###############################################################################
# Transform High-Volume drink columns to log scale
###############################################################################
import numpy as np

# Make a copy so we don’t lose original sales values
model_df_high_log = model_df_high.copy()

# Replace the target columns in model_df_high_log with their log1p (log(x+1)) version
for drink_name in drink_columns_high:
    model_df_high_log[drink_name] = np.log1p(model_df_high_log[drink_name])

# Now define Y in the log-scale
Y_high_log = model_df_high_log[drink_columns_high].values

# Proceed with train_test_split on X_high and Y_high_log
Xh_train, Xh_test, Yh_train_log, Yh_test_log = train_test_split(
    X_high,
    Y_high_log,
    test_size=0.2,
    random_state=42
)


# rf_high = RandomForestRegressor(n_estimators=100, random_state=42)
# multi_high = MultiOutputRegressor(rf_high)
# multi_high.fit(Xh_train, Yh_train)
# Yh_pred = multi_high.predict(Xh_test)

# # Evaluate
# mae_per_drink = []
# r2_per_drink = []
# for i, dcol in enumerate(drink_columns_high):
#     mae_i = mean_absolute_error(Yh_test[:, i], Yh_pred[:, i])
#     r2_i = r2_score(Yh_test[:, i], Yh_pred[:, i])
#     mae_per_drink.append(mae_i)
#     r2_per_drink.append(r2_i)
#     print(f"Drink '{dcol}' => MAE: {mae_i:.2f}, R²: {r2_i:.2f}")

# print(f"\n[High-Volume] Average MAE: {np.mean(mae_per_drink):.2f}")
# print(f"[High-Volume] Average R²: {np.mean(r2_per_drink):.2f}")

# (B) Mid/Low Volume Model
print("\n===== MID/LOW VOLUME MODEL =====")
model_df_mid_low = final_df.dropna(subset=feature_columns).copy()
X_mid_low = model_df_mid_low[feature_columns].values
Y_mid_low = model_df_mid_low[drink_columns_mid_low].values

###############################################################################
# Transform Mid/Low-Volume drink columns to log scale
###############################################################################
model_df_mid_low_log = model_df_mid_low.copy()

for drink_name in drink_columns_mid_low:
    model_df_mid_low_log[drink_name] = np.log1p(model_df_mid_low_log[drink_name])

Y_mid_low_log = model_df_mid_low_log[drink_columns_mid_low].values

Xm_train, Xm_test, Ym_train_log, Ym_test_log = train_test_split(
    X_mid_low,
    Y_mid_low_log,
    test_size=0.2,
    random_state=42
)


# rf_mid_low = RandomForestRegressor(n_estimators=100, random_state=42)
# multi_mid_low = MultiOutputRegressor(rf_mid_low)
# multi_mid_low.fit(Xm_train, Ym_train)
# Ym_pred = multi_mid_low.predict(Xm_test)

# # Evaluate
# mae_per_drink = []
# r2_per_drink = []
# for i, dcol in enumerate(drink_columns_mid_low):
#     mae_i = mean_absolute_error(Ym_test[:, i], Ym_pred[:, i])
#     r2_i = r2_score(Ym_test[:, i], Ym_pred[:, i])
#     mae_per_drink.append(mae_i)
#     r2_per_drink.append(r2_i)
#     print(f"Drink '{dcol}' => MAE: {mae_i:.2f}, R²: {r2_i:.2f}")

# print(f"\n[Mid/Low Volume] Average MAE: {np.mean(mae_per_drink):.2f}")
# print(f"[Mid/Low Volume] Average R²: {np.mean(r2_per_drink):.2f}")


Sales Data Sample:
              Datum Betaaltype          Product  Aantal
0  26-09-2022 00:53  Omzet pin  Buitenlands mix       1
1  26-09-2022 00:52  Omzet pin        Heineken        1
2  26-09-2022 00:31  Omzet pin             Fris       1
3  26-09-2022 00:31  Omzet pin     Sourcy blauw       1
4  26-09-2022 00:25  Omzet pin     Sourcy blauw       1

Event & Weather Data Sample:
   Unnamed: 0            Datum_uur    Omzet       Datum     Tmax     Tmin  \
0          28  2022-07-09 14:00:00     9.00  2022-07-09  21.6 °C  15.3 °C   
1          29  2022-07-09 16:00:00   182.35  2022-07-09  21.6 °C  15.3 °C   
2          30  2022-07-09 17:00:00   767.10  2022-07-09  21.6 °C  15.3 °C   
3          31  2022-07-09 18:00:00  1933.70  2022-07-09  21.6 °C  15.3 °C   
4          32  2022-07-09 19:00:00  2567.40  2022-07-09  21.6 °C  15.3 °C   

  Neerslag Max Windstoot first_event_date_start  aantal_tickets  \
0   0,0 mm     30.6 km/u    2022-07-09 16:00:00          1319.0   
1   0,0 mm     30.

In [85]:
from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score

###############################################################################
# 1. Define Sample Weights for High Volume
###############################################################################
# The shape of Yh_train_log is [n_samples, n_drinks_high].
# Exponentiate back to original scale to see actual volumes:
Yh_train_orig = np.expm1(Yh_train_log)  # shape: (num_rows, num_high_drinks)

# Example approach: use the SUM of all high-volume drinks in that row as the row weight
row_sum_high = Yh_train_orig.sum(axis=1)  # shape: (num_rows,)
# Optionally scale or clamp to avoid zero/huge weights:
sample_weights_high = np.maximum(1, row_sum_high / 100.0)  # or any scaling you prefer
# sample_weights_high = np.sqrt(1, row_sum_high / 100.0)  # or any scaling you prefer


print("Sample weights (High):", sample_weights_high[:10])  # quick peek

###############################################################################
# 2. Fit CatBoost with sample_weight
###############################################################################
cat_high = CatBoostRegressor(
    iterations=160,
    learning_rate=0.07,
    depth=4,
    random_seed=42,
    verbose=0
)
multi_cat_high = MultiOutputRegressor(cat_high)

multi_cat_high.fit(
    Xh_train,
    Yh_train_log,
    sample_weight=sample_weights_high
)

# Predict in log scale
Yh_pred_log = multi_cat_high.predict(Xh_test)

# Exponentiate back to original scale
Yh_pred = np.expm1(Yh_pred_log)
Yh_test_orig = np.expm1(Yh_test_log)

# Evaluate
mae_list, r2_list = [], []
for i, drink_name in enumerate(drink_columns_high):
    mae_i = mean_absolute_error(Yh_test_orig[:, i], Yh_pred[:, i])
    r2_i = r2_score(Yh_test_orig[:, i], Yh_pred[:, i])
    mae_list.append(mae_i)
    r2_list.append(r2_i)
    print(f"[Weighted CAT-High] Drink '{drink_name}' => MAE: {mae_i:.2f}, R²: {r2_i:.2f}")

print(f"\n[Weighted CAT-High] Avg MAE: {np.mean(mae_list):.2f}")
print(f"[Weighted CAT-High] Avg R²: {np.mean(r2_list):.2f}")



# Mid/Low Volume
###############################################################################
# Fit mid/low model on log scale, then exponentiate predictions
###############################################################################
# cat_mid = CatBoostRegressor(
#     iterations=100,
#     learning_rate=0.01,
#     depth=6,
#     random_seed=42,
#     verbose=0
# )

# multi_cat_mid = MultiOutputRegressor(cat_mid)
# multi_cat_mid.fit(Xm_train, Ym_train_log)

# Ym_pred_log = multi_cat_mid.predict(Xm_test)
# Ym_pred = np.expm1(Ym_pred_log)
# Ym_test_orig = np.expm1(Ym_test_log)

# mae_list, r2_list = [], []
# for i, drink_name in enumerate(drink_columns_mid_low):
#     # Evaluate predictions vs actual in original space
#     mae_i = mean_absolute_error(Ym_test_orig[:, i], Ym_pred[:, i])
#     r2_i = r2_score(Ym_test_orig[:, i], Ym_pred[:, i])
#     mae_list.append(mae_i)
#     r2_list.append(r2_i)
#     print(f"[CAT-Mid] Drink '{drink_name}' => MAE: {mae_i:.2f}, R²: {r2_i:.2f}")

# print(f"\n[CAT-Mid] Avg MAE: {np.mean(mae_list):.2f}")
# print(f"[CAT-Mid] Avg R²: {np.mean(r2_list):.2f}")




Sample weights (High): [ 8.08 56.1  26.62 57.3  47.33 50.69 59.69 24.02  1.   44.45]
[Weighted CAT-High] Drink 'Desperados' => MAE: 53.98, R²: 0.19
[Weighted CAT-High] Drink 'Fris' => MAE: 225.15, R²: 0.74
[Weighted CAT-High] Drink 'Heineken' => MAE: 340.45, R²: 0.70
[Weighted CAT-High] Drink 'RedBull' => MAE: 89.31, R²: 0.18
[Weighted CAT-High] Drink 'Shot' => MAE: 80.12, R²: 0.24
[Weighted CAT-High] Drink 'Sterk Mix' => MAE: 362.05, R²: -0.38

[Weighted CAT-High] Avg MAE: 191.85
[Weighted CAT-High] Avg R²: 0.28


In [81]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_predicted_vs_actual_logscale(Y_true_log, Y_hat_log, drink_cols, num_plots=4):
    """
    Plots predicted vs. actual (in *original* scale) for the first 'num_plots' drinks
    in the list 'drink_cols', given that Y_true_log and Y_hat_log
    are in log scale (log1p).
    """
    n_drinks = len(drink_cols)
    n_plots = min(num_plots, n_drinks)

    plt.figure(figsize=(5 * n_plots, 4))
    for i in range(n_plots):
        # Convert from log-scale back to original scale
        actual_vals = np.expm1(Y_true_log[:, i])  # exp(Y) - 1
        pred_vals   = np.expm1(Y_hat_log[:, i])

        plt.subplot(1, n_plots, i + 1)
        sns.scatterplot(x=actual_vals, y=pred_vals, alpha=0.5)

        max_val = max(actual_vals.max(), pred_vals.max())
        plt.plot([0, max_val], [0, max_val], color='red', linestyle='--')
        plt.xlabel("Actual (original scale)")
        plt.ylabel("Predicted (original scale)")
        plt.title(f"{drink_cols[i]}: Pred vs. Actual")
    plt.tight_layout()
    plt.show()


In [62]:
# List the columns you want
cols = ['Fris', 'Heineken', 'Desperados', 'RedBull']

# Calculate median for each of these
median_values = final_df[cols].median()


for drink_name, med_val in median_values.items():
    print(f"Median sales for {drink_name}: {med_val}")


Median sales for Fris: 645.0
Median sales for Heineken: 1433.5
Median sales for Desperados: 122.5
Median sales for RedBull: 113.5


## AFWIJKINGEN PER EVENT PREDICTION

Heinken is ongeveer 24% van de mediaan <br>
Fris is ongeveer 35% van de mediaan <br>
Desperados is ongeveer 44% van de mediaan <br>
RedBull is ongeveer 78% van de mediaan

